# Programación 2 - TUIA
---
## Trabajo Práctico - Tabla Hash




- Comisión: 2
- (integrante 1): Avecilla Tomás

In [ ]:
from typing import Any

class Clave():
  """
  Almacena la clave real a ser hasheada,
  usando la representación que se desee,
  y encapsula el algoritmo de hash elegido.
  """

  def __init__(self, key: str):
    self.key = key

  def to_int(self) -> int:
    """
    Esta función convierte la clave en un número entero de forma simple.
    """
    hash_val = 0
    for char in str(self.key):
        hash_val = (hash_val * 256 + ord(char)) % 39977 # 256 Base de ASCII y numero primo cercano a 40000
    return hash_val

  def __eq__(self, other: Any) -> bool:
    """
    Decide si dos claves son iguales
    """
    return self.key == other.key


In [ ]:
class _Nodo2:
  """
  Implementacion de nodos para tablas hash encadenadas.
  """
  def __init__(self, key = None, data = None, next = None):
      self.key = key
      self.data = data
      self.next = None

class ListaEnlazada:
    """
    Implementacion de una Lista Enlazada para trabajar las colisiones
    de una Tabla Hash
    """
    def __init__(self):
        self.head = None
        self.len = 0

    def append(self, key: Clave, dato: Any) -> None:
        if not self.head:
            self.head = _Nodo2(key, dato)
        else:
          indice = self.head
          while indice.next:
            indice = indice.next
          indice.next = _Nodo2(key, dato)
        self.len += 1

    def insert(self, key: Clave, pos: int, dato: Any) -> None:
        if pos < 1 or pos > self.len + 1:
          print("Posición fuera de rango.")
          return
        if pos == 1:
          nodo = _Nodo2(key, dato, self.head)
          self.head = nodo
        else:
          contador = 1
          actual = self.head
          while contador < pos - 1:
            actual = actual.next
            contador += 1
          nuevo_nodo = _Nodo2(key, dato, actual.next)
          actual.next = nuevo_nodo
        self.len += 1

    def index(self, search_key: Clave):
        actual = self.head
        index = 0
        while actual:
            if actual.key == search_key:
                return index
            actual = actual.next
            index += 1
        return None

    def remove(self, key: Clave) -> None:
      """
      Remueve la primera aparicion de la clave dada
      """
      if not self.head:
        return None
      if self.head.key == key:
        self.head = self.head.next
        self.len -= 1
        return
      else:
        anterior = self.head
        actual = anterior.next
        while actual is not None and actual.key != key:
          anterior = actual
          actual = anterior.next
        if actual == None:
          print("El elemento no esta en la lista")
          return
        anterior.next = actual.next
        self.len -= 1

    def pop(self, pos: int) -> Any:
        if pos < 1 or pos > self.len:
          return None
        if pos == 1:
          key = self.head.key
          dato = self.head.dato
          self.head = self.head.next
        else:
          contador = 1
          actual = self.head
          while contador < pos - 1:
              actual = actual.next
              contador += 1
          key = actual.next.clave
          dato = actual.next.dato
          actual.next = actual.next.next
        self.len -= 1
        return key, dato

    def __len__(self):
      return self.len

    def __str__(self):
      if not self.head:
        return "Este Bucket está vacío"
      else:
        iterador = self.head
        lista = []
        while iterador:
            lista.append(f"{iterador.key} : {iterador.dato}")
            iterador = iterador.next
        return " --> ".join(lista)


In [ ]:
# No es necesario cambiar nada en esta celda! Esta celda solo define la interfaz
# del TAD Diccionario.
from typing import Any

class Diccionario():
  """
  Interfaz del TAD Diccionario
  - insert(key, value) - Inserta un elemento con clave key y valor value en el diccionario.
      Si la clave ya se encuentra en el diccionario, debe reemplazar el value anterior por el nuevo.
  - search(key) - Devuelve la el value asociado con la clave key, o muestra un mensaje de error si la clave no se encuentra.
  - delete(key) - Elimina el elemento con clave key del diccionario.
  """

  def insert(self, key: Clave, data: Any) -> None:
    pass

  def search(self, key: Clave) -> Any:
    pass

  def delete(self, key: Clave) -> None:
    pass


In [ ]:
class HashTable(Diccionario):
  """
  Implementación de Diccionario con tabla hash
  de tamaño fijo y resolución de colisiones
  por encadenamiento
  """

  def __init__(self, size: int):
    """
    El constructor inicializa una tabla de tamaño `size`.
    """
    self.size = size
    self.table = [ListaEnlazada() for i in range(size)]
    self.len = 0

  def hash_function(self, key: Clave) -> int:
    return key.to_int() % self.size

  def insert(self, key: Clave, data: Any) -> None:
    """
    Implementación del método insertar con encadenamiento.
    """
    index = self.hash_function(key)
    bucket = self.table[index]
    bucket.append(key, data)
    self.len += 1

  def search(self, key: Clave) -> Any | None:
    """
    Implementacion del metodo buscar.
    """
    index = self.hash_function(key)
    bucket = self.table[index]
    if bucket.head is None:
      return None
    nodo_actual = bucket.head
    while nodo_actual:
      if nodo_actual.key == key:
        return nodo_actual.data
      nodo_actual = nodo_actual.next
    return None

  def delete(self, key: Clave) -> None:
    """
    Implementacion del metodo delete.
    """
    index = self.hash_function(key)
    bucket = self.table[index]
    if bucket.head is None:
      return None
    nodo_actual = bucket.head
    while nodo_actual:
      if nodo_actual.key == key:
        self.len -= 1
        return bucket.remove(key)
      nodo_actual = nodo_actual.next
    return None

  def __len__(self) -> int:
    """
    Este método sobrecarga la función `len`.
    Debe devolver la cantidad de elementos insertados en la tabla.
    Si la tabla esta vacía, devuelve cero.
    """
    return self.len

  def _load_factor(self) -> float:
    """
    Este método debe devolver el factor de carga de la tabla
    """
    factor_de_carga = self.len / self.size
    return factor_de_carga

  def _longest_list(self) -> int:
    """
    Este método debe devolver la longitud de la lista mas larga de la tabla
    """
    longitud = 0
    for bucket in self.table:
      if bucket.len > longitud:
        longitud = bucket.len
    return longitud

  def __str__(self) -> str:
    """
    Mostrar la cantidad de elementos de la tabla (__len__), el factor de carga (_load_factor)
    y la longitud de la lista mas larga (_longest_list)
    """
    cantidad = str(self.len)
    factor = str(self._load_factor())
    mayor_lista = str(self._longest_list())
    return f"Cantidad de elementos: {cantidad}.\nFactor de carga: {factor}.\nLongitud de la lista mas larga: {mayor_lista}"

Código de prueba propio


In [ ]:
from random import choice, randrange

def generar_clave_random() -> Clave:
  """
  Genera una clave alfanumerica de 10 digitos al azar
  """
  chars = "1234567890QWERTYUIOPASDFGHJKLZXCVBNM"
  string = "".join(choice(chars) for _ in range(10))
  return Clave(string)

def generar_valor_random() -> int:
  """
  Devuelve un valor al azar entre 0 y 100 millones
  """
  return randrange(0, 100_000_000)

# Crea una instancia de la tabla hash con un tamaño específico
tamaño_tabla = 40000
tabla = HashTable(tamaño_tabla)

# Inserta una cantidad significativa de elementos en la tabla hash
for i in range(53783):
    clave = generar_clave_random()
    valor = generar_valor_random()
    tabla.insert(clave, valor)

# Realiza búsquedas y eliminaciones
clave_a_buscar = generar_clave_random()
valor_a_buscar = generar_valor_random()
print(valor_a_buscar)
tabla.insert(clave_a_buscar, valor_a_buscar)
resultado1 = tabla.search(clave_a_buscar)
print(resultado1)
tabla.delete(clave_a_buscar)
r = tabla.search(clave_a_buscar)
print(r)
print(tabla)

10658047
10658047
None
Cantidad de elementos: 53783.
Factor de carga: 1.344575.
Longitud de la lista mas larga: 9


## Código de Prueba

La siguiente celda deberia funcionar como un test básico de funcionamiento. Si su implementación tiene sentido, debería poder correr la siguiente celda sin problemas. En caso de que la prueba funcione, ejecutar el siguiente código no imprime nada. En caso de problemas, mostrara un error.

**Nota**: El funcionamiento de este código no quiere decir que la solución sea correcta. La prueba es muy básica ya que solo prueba insertar y eliminar un elemento.

**Nota: La instrucción `assert`**: El uso de assert en Python nos permite realizar comprobaciones. Si la expresión contenida dentro del mismo es False, se lanzará un error, mas concretamente un `AssertionError`. Por ejemplo, `assert 1==2` nos daría un `AssertionError`. Puede leer mas sobre esta instrucción en [el libro de Python](https://ellibrodepython.com/assert-python).



**ATENCION: NO MODIFIQUE ESTA CELDA** Si desea hacer más pruebas, hagalo en una celda nueva.

In [ ]:
from random import choice, randrange

def generar_clave_random() -> Clave:
  """
  Genera una clave alfanumerica de 10 digitos al azar
  """
  chars = "1234567890QWERTYUIOPASDFGHJKLZXCVBNM"
  string = "".join(choice(chars) for _ in range(10))
  return Clave(string)

def generar_valor_random() -> int:
  """
  Devuelve un valor al azar entre 0 y 100 millones
  """
  return randrange(0, 100_000_000)

def prueba_de_funcionamiento(diccionario: Diccionario):
  # assert rompe el programa si la condición es falsa
  # esto es útil para verificar cosas que deberían ser ciertas

  clave = generar_clave_random()
  valor = generar_valor_random()

  diccionario.insert(clave, valor)

  assert len(diccionario) == 1

  assert diccionario.search(clave) == valor

  diccionario.delete(clave)

  assert len(diccionario) == 0

# Para probar esto, primero implemente to_int en Celda y la clase HashTable
prueba_de_funcionamiento(HashTable(100))